##### Parte 1 > Realizando import's e iniciando pyspark

In [9]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StringType
import random
from pyspark.sql.functions import *

In [6]:
# Nesta etapa, adicoine código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. 
# Carregue-o para dentro de um dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

spark = SparkSession.builder.master("local[*]").appName("Tarefa").getOrCreate()
df_nomes = spark.read.csv('../Tarefa3/nomes_aleatorios.txt')
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



##### Parte 2 > Renomear coluna, imprimir o schema e mostrar 10 linhas

In [8]:
df_nomes = df_nomes.withColumnRenamed('_c0', 'Nomes')
df_nomes.printSchema()
df_nomes.show(10)


root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



##### Parte 3 > Adicionar nova coluna chamada Escolaridade

In [18]:
escolaridade = ['Fundamental', 'Médio', 'Superior']

df_nomes = df_nomes.withColumn('Escolaridade', udf(f=lambda: escolaridade[random.randint(0,2)],returnType=StringType())())

##### Parte 4 > Adicionar países da América do Sul


In [20]:
paises = ["Brasil", "Argentina", "Colômbia", "Peru", "Venezuela", "Chile", "Equador",
            "Bolívia", "Paraguai", "Uruguai", "Guiana", "Suriname", "Guiana Francesa"]

df_nomes = df_nomes.withColumn('Pais', udf(f=lambda: paises[random.randint(0,12)],returnType=StringType())())

##### Parte 5 > Adicione nova coluna chamada AnoNascimento

In [26]:
df_nomes = df_nomes.withColumn("AnoNascimento", expr("CAST(FLOOR(RAND() * (66) + 1945) AS INT)"))

##### Parte 6 > Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século

In [27]:
df_select = df_nomes.select('*').where(df_nomes.AnoNascimento >= 2000)
df_select.show(10)

+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|   Frances Bennet|       Médio|Guiana Francesa|         2003|
|Berniece Ornellas|    Superior|        Equador|         2002|
|   Kara Mcelwaine| Fundamental|           Peru|         2010|
|     Charles Hill|       Médio|       Suriname|         2008|
|         Mary Lee| Fundamental|         Brasil|         2008|
| Flossie Anderson| Fundamental|      Venezuela|         2005|
|    Milton Dillon|    Superior|        Equador|         2000|
|     James Barton| Fundamental|       Colômbia|         2008|
|   Ashley Trosper| Fundamental|       Paraguai|         2008|
|       April Ward|       Médio|       Colômbia|         2010|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



##### Parte 7 > Usando Spark SQL repita o processo da Pergunta 6.

In [29]:
df_nomes.createOrReplaceTempView('pessoas')
spark.sql('SELECT * FROM pessoas WHERE AnoNascimento >= 2000').show(10)


+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|   Frances Bennet|    Superior|        Uruguai|         2003|
|Berniece Ornellas|    Superior|        Bolívia|         2002|
|   Kara Mcelwaine|       Médio|      Venezuela|         2010|
|     Charles Hill|    Superior|          Chile|         2008|
|         Mary Lee|       Médio|Guiana Francesa|         2008|
| Flossie Anderson|       Médio|       Suriname|         2005|
|    Milton Dillon| Fundamental|        Bolívia|         2000|
|     James Barton| Fundamental|       Colômbia|         2008|
|   Ashley Trosper|    Superior|        Equador|         2008|
|       April Ward|    Superior|      Argentina|         2010|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



##### Parte 8 > Conte o número de pessoas que são da geração Millennials


In [30]:
m = df_nomes.select('*').where(df_nomes.AnoNascimento.between(1980, 1994)).count()
print(m)


2272548


##### Parte 9 > Repita o processo da Pergunta 8 utilizando Spark SQL

In [32]:
spark.sql('SELECT COUNT(*) as Contagem FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994').show()

+--------+
|Contagem|
+--------+
| 2272548|
+--------+



##### Parte 10 > Obtenha a quantidade de pessoas de cada país para uma das gerações

In [35]:
Geracao_Z = spark.sql("SELECT Pais, 'Baby Boomers' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1944 AND 1964\
                        GROUP BY Pais\
                        ORDER BY Pais, Geracao, Quantidade").show()

+---------------+------------+----------+
|           Pais|     geracao|Quantidade|
+---------------+------------+----------+
|      Argentina|Baby Boomers|    233226|
|        Bolívia|Baby Boomers|    233545|
|         Brasil|Baby Boomers|    233038|
|          Chile|Baby Boomers|    232761|
|       Colômbia|Baby Boomers|    232683|
|        Equador|Baby Boomers|    232941|
|         Guiana|Baby Boomers|    232991|
|Guiana Francesa|Baby Boomers|    233901|
|       Paraguai|Baby Boomers|    233619|
|           Peru|Baby Boomers|    233205|
|       Suriname|Baby Boomers|    233042|
|        Uruguai|Baby Boomers|    232501|
|      Venezuela|Baby Boomers|    232429|
+---------------+------------+----------+

